In [1]:
import extFunc as ext
import numpy as np
from importlib import import_module
import pandas as pd 
import cobra
import optlang
import copy 
import random


In [2]:
#Load Models if not already in RAM

#nameList = ['iJN1462_GLC_UR10_9.xml' , 'iJN1462_GLC_UR6_3.xml', 'iJN1462_GLN_UR5_1.xml', 'iJN1462_GLC_UR7_3.xml' , 'iJN1462_OCT_UR3_4.xml']
nameList = ['iJN1462_GLC_UR6_3.xml' , 'iJN1462_OCT_UR3_4.xml']
#nameList = ['iJN1462_OCT_UR3_4.xml']

if 'modelDict' not in locals():
    modelDict = ext.ImportFunction(nameList)

    

Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled


### Implementation of Model comparison sorted by C-flux

In [15]:
if 'glc_oct_comp'  not in locals():
    glc_oct_comp = ext.ModelComparison(modelDict=modelDict)
    
glc_oct_comp.multiModellSummary(sortKrit='C-Flux',fvaDIr=.99)


Uptake                          \
                     metabolite      reaction      flux   
iJN1462_GLC_UR6_3 0   glc__D[e]   EX_glc__D_e  6.300000   
                  1  pqqA_kt[c]  SK_pqqA_kt_c  0.000137   
                  2      na1[e]      EX_na1_e  0.002168   
                  3      zn2[e]      EX_zn2_e  0.001734   
                  4      so4[e]      EX_so4_e  0.114504   
iJN1462_OCT_UR3_4 0     octa[e]     EX_octa_e  3.400000   
                  1  pqqA_kt[c]  SK_pqqA_kt_c  0.000065   
                  2      nh4[e]      EX_nh4_e  3.100000   
                  3     PHAg[c]     SK_PHAg_c  1.117172   
                  4      zn2[e]      EX_zn2_e  0.000820   

                                                                       \
                                                                range   
iJN1462_GLC_UR6_3 0                          (6.299999999993848, 6.3)   
                  1  (0.00013728582221364972, 0.00013728582221377494)   
                  2     (0.0021682541068900178, 0.002168254106894571)   
                  3     (0.0017342339066181663, 0.001734233906621996)   
                  4        (0.11450376330405945, 0.11450376337845493)   
iJN1462_OCT_UR3_4 0                                        (3.4, 3.4)   
                  1    (6.494637531958918e-05, 6.494637531969528e-05)   
                  2                          (3.100000000000168, 3.1)   
                  3                         (0.0, 2.1332831168002007)   
                  4     (0.0008204212523555279, 0.000820421252359735)   

                                             Secretion                   \
                    C-Number     C-Flux     metabolite         reaction   
iJN1462_GLC_UR6_3 0        6  99.994916         co2[e]         EX_co2_e   
                  1       14   0.005084       5drib[c]       DM_5drib_c   
                  2        0   0.000000        amob[c]        DM_amob_c   
                  3        0   0.000000      doxopa[c]      DM_doxopa_c   
                  4        0   0.000000  tripeptide[c]  DM_tripeptide_c   
iJN1462_OCT_UR3_4 0        8  99.996657     C80aPHA[c]     DM_C80aPHA_c   
                  1       14   0.003343         co2[e]         EX_co2_e   
                  2        0   0.000000    R_3hocta[e]    EX_R_3hocta_e   
                  3        0   0.000000          ac[e]          EX_ac_e   
                  4        0   0.000000       5drib[c]       DM_5drib_c   

                                \
                          flux   
iJN1462_GLC_UR6_3 0 -13.008062   
                  1  -0.000412   
                  2  -0.000137   
                  3  -0.000137   
                  4  -0.000137   
iJN1462_OCT_UR3_4 0  -1.117172   
                  1  -3.171260   
                  2  -0.377543   
                  3  -0.171264   
                  4  -0.000195   

                                                                        \
                                                                 range   
iJN1462_GLC_UR6_3 0          (-13.008062493678919, -13.00806249313996)   
                  1    (-0.0004118574707388604, -0.000411857466640949)   
                  2  (-0.00013728582282382935, -0.00013728582221364...   
                  3  (-0.00013728582256783694, -0.0001372858222136497)   
                  4  (-0.00013728582221378323, -0.0001372858222136497)   
iJN1462_OCT_UR3_4 0                         (-1.5447912225086302, 0.0)   
                  1             (-15.47157493597859, 4.13814641690933)   
                  2                         (-1.5342104607162232, 0.0)   
                  3                           (-7.74757879745532, 0.0)   
                  4  (-0.00045462463094977245, -0.00019483912595876...   

                                         
                    C-Number     C-Flux  
iJN1462_GLC_UR6_3 0        1  99.965184  
                  1        5   0.015825  
                  2       15   0.015825  
                  3        3  

### Model comparison for different objectives sorted by C-flux 

In [16]:

Case    = 'iJN1462_OCT_UR3_4'
nameList = ['iJN1462_OCT_UR3_4.xml','iJN1462_GLC_UR6_3.xml']
modelDict = ext.ImportFunction(nameList)

with modelDict[Case] as testModel:
    ATP_objective = testModel.problem.Objective(
        ## Ask Franzi which atp related reactions should be considered
        +1 * testModel.reactions.PYK.flux_expression        #   Pyruvate kinase
        +1 * testModel.reactions.ATPS4rpp.flux_expression   #   ATP synthase
        +1 * testModel.reactions.URIDK2r.flux_expression    #   Uridylate kinase (dUMP)
        +1 * testModel.reactions.PPK.flux_expression        #   Polyphosphate kinase
        +1 * testModel.reactions.PPK2.flux_expression       #   Polyphosphate kinase
        +1 * testModel.reactions.PGK.flux_expression        #  Phosphoglycerate kinase
        , direction='max')
    testModel.objective = ATP_objective
    ATPsolution     =  testModel.optimize()
    pFBAsolution    = cobra.flux_analysis.loopless.loopless_solution(modelDict[Case])

solutionDict = {
                'fba'                   :  modelDict[Case].optimize() ,
                'pfba'                  :  cobra.flux_analysis.pfba(modelDict[Case]),
                'CylceFreeFlux_algo'    :  cobra.flux_analysis.loopless.loopless_solution(modelDict[Case]),
                'ATP_maximization'      :  ATPsolution,
                'pFBA(ATP_maximization)':  pFBAsolution,
                #'geom_fba'             :  cobra.flux_analysis.geometric_fba(modelDict[Case]),
                #'looples_cla'          :  cobra.flux_analysis.loopless.add_loopless(modelDict[Case]).optimize(),
                }

single_Model_comp = ext.ModelComparison(modelDict=modelDict)


In [17]:
# Comment an Franzi, wenn summary Methode aufgerufen wir, wird automatisch pFBA ausgeführt daher untershiede im ergebniss
# zwischen fba und pfba, kurzum: gba result liefert gar keine DM_80aPHA_c
# fba -->       Maximierung der Zielfunktion, thats ix
# pfba -->      Maximierung der Zielfunktion, und minimierung der Summe aller Flüsse
# geometric --> Geometric FBA finds a unique optimal flux distribution which is central to the range of possible fluxes.
# pfba --> Maximierung der Zielfunktion, und minimierung der Summe aller Flüsse

single_Model_comp.singleModellSummary(solutionDict,nrRea=3,sortKrit='C-Flux')

Process SpawnPoolWorker-68:
Process SpawnPoolWorker-67:
Process SpawnPoolWorker-66:
Process SpawnPoolWorker-64:
Process SpawnPoolWorker-65:
Traceback (most recent call last):
  File "/Users/ivanlizat/opt/anaconda3/envs/hiwi/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/ivanlizat/opt/anaconda3/envs/hiwi/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/ivanlizat/opt/anaconda3/envs/hiwi/lib/python3.9/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/Users/ivanlizat/opt/anaconda3/envs/hiwi/lib/python3.9/multiprocessing/queues.py", line 365, in get
    with self._rlock:
  File "/Users/ivanlizat/opt/anaconda3/envs/hiwi/lib/python3.9/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
Traceback (most recent call last):
  File "/Users/ivanlizat/opt/anaconda3/envs/hiwi/lib/python3.9/multiprocess

KeyboardInterrupt: 

In [6]:
nameList = ['iJN1462_OCT_UR3_4.xml']
modelDict = ext.ImportFunction(nameList)
solutionDict = {
                'pfba'               :  cobra.flux_analysis.pfba(modelDict['iJN1462_OCT_UR3_4']),
                }

single_Model_comp = ext.ModelComparison(modelDict=modelDict)

In [7]:
env_conditions = {r.id:(r.lower_bound, r.upper_bound)
 for r in modelDict['iJN1462_OCT_UR3_4'].reactions
 if (r.lower_bound not in [-1000, 0]) or (r.upper_bound not in [0, 1000])}
env_conditions


{'EX_ca2_e': (-10.0, 1000.0),
 'EX_cl_e': (-10.0, 1000.0),
 'EX_co2_e': (-100.0, 1000.0),
 'EX_cobalt2_e': (-10.0, 1000.0),
 'EX_cu2_e': (-10.0, 1000.0),
 'EX_fe2_e': (-10.0, 1000.0),
 'EX_h2o_e': (-100.0, 1000.0),
 'EX_h_e': (-100.0, 1000.0),
 'EX_hco3_e': (-10.0, 1000.0),
 'EX_k_e': (-10.0, 1000.0),
 'EX_mg2_e': (-10.0, 1000.0),
 'EX_mn2_e': (-10.0, 1000.0),
 'EX_mobd_e': (-10.0, 1000.0),
 'EX_na1_e': (-10.0, 1000.0),
 'EX_nh4_e': (-3.1, 1000.0),
 'EX_ni2_e': (-10.0, 1000.0),
 'EX_o2_e': (-13.5, 1000.0),
 'EX_octa_e': (-3.4, -3.4),
 'EX_pi_e': (-10.0, 1000.0),
 'EX_sel_e': (-10.0, 1000.0),
 'EX_so4_e': (-10.0, 1000.0),
 'EX_tungs_e': (-10.0, 1000.0),
 'EX_zn2_e': (-10.0, 1000.0),
 'SK_pqqA_kt_c': (-1.0, 0.0),
 'ATPM': (0.92, 0.92)}

In [8]:
env_conditions = {r.id:(r.lower_bound, r.upper_bound)
 for r in modelDict['iJN1462_OCT_UR3_4'].demands
 if (r.lower_bound not in [0, 0]) or (r.upper_bound not in [0, 0])}

env_conditions


{'DM_5drib_c': (0.0, 1000.0),
 'DM_amob_c': (0.0, 1000.0),
 'DM_C100aPHA_c': (0, 1000),
 'DM_C100pPHA_c': (0, 1000),
 'DM_C120aPHA_c': (0, 1000),
 'DM_C60aPHA_c': (0, 1000),
 'DM_C60pPHA_c': (0, 1000),
 'DM_C80aPHA_c': (0, 1000),
 'DM_C80pPHA_c': (0, 1000),
 'DM_acmum6p_c': (0.0, 1000.0),
 'DM_ppi50_c': (0.0, 1000.0),
 'DM_acgam_c': (0.0, 1000.0),
 'DM_doxopa_c': (0.0, 1000.0),
 'DM_tripeptide_c': (0.0, 1000.0)}